#### Sample code for NER 

In [475]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import pycrfsuite

In [476]:
'''
Here in this notebook, I am doing the experiment using CONLL corpus 
I have to clean the data a bit

and have then extracted features for organization 
for all the other words in the coprus , i put them in OTHER category and then doing the prediction using SVM and logistic 
regression

Here the approach to follow is
make 4 classes for person, location, organization, misc
make 5th class for others ( like verb , CD )
and then do classification for all these entities in one go


but the question is - will the feature for organization be similar to name or location.
Well most of them are .. so for non relevant features we can put NA

else look for some advance approach, like adding random forest , e.g
https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words 
'''

'\nHere in this notebook, I am doing the experiment using CONLL corpus \nI have to clean the data a bit\n\nand have then extracted features for organization \nfor all the other words in the coprus , i put them in OTHER category and then doing the prediction using SVM and logistic \nregression\n\nHere the approach to follow is\nmake 4 classes for person, location, organization, misc\nmake 5th class for others ( like verb , CD )\nand then do classification for all these entities in one go\n\n\nbut the question is - will the feature for organization be similar to name or location.\nWell most of them are .. so for non relevant features we can put NA\n\nelse look for some advance approach, like adding random forest , e.g\nhttps://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words \n'

In [477]:
NER_train_Corpus = open ( "C:\\share\\Co-nll\\eng.testa" , 'r')
#read the complete corpus
corpus= NER_train_Corpus.read()

#separate in sentences
sentences_in_corpus = sent_tokenize(corpus)

#separate the words in each sentence
sentences_with_words = []
for sent in sentences_in_corpus:
    sentences_with_words.append( word_tokenize(sent) )

#do some cleaning of corpus
#1. remove 'O' , 'O' from start
#2. remove sentence with 1 word
count = 0
length = len(sentences_with_words)
print length
while (count < length):
    words = sentences_with_words[count]

    if words[0] == '.' : 
        del sentences_with_words[count]
        length = length - 1
    elif (words[0] == 'O' and words[1] == 'O' ): 
        del sentences_with_words[count][0]
        del sentences_with_words[count][0]
        count = count + 1
    else :
        count = count + 1

NER_train_Corpus.close()

3975


In [478]:
def is_first_letter_in_word_capitalized( word ):
    if not word.islower() and not word.isupper():    
        return 1
    else:
        return 0

In [479]:
'''
Features 

Word features/Lexical features: 
    1) Bag of words - Tokens in the window c = (xi−2, xi−1, xi , xi+1, xi+2)
        1) current word 
        2) previous two words,
        3) next two words

Syntactic features
    1) POS tags for xi−1, xi , xi+1
    2) IOB tags for xi−1, xi , xi+1

Orthographic features :
    1) capitalization pattern in the window c 
    E.g . Jenny --> Xxxx
        . IL-2 --> XX-#
    But i have taken sequence of only first alphabet in each word, X if capital , x if small
    2) xi word type 
        1) all-capitalized - > are all letters capitalized
        2) is-capitalized -> is the first letter capitalized
        5) is it the first word in sentence
        6) is it the last word in sentence
    3) capitalization pattern in the window c 


### Not used features
Word features :
    1) Trigger words - (e.g. Inc., Co., Mr.).

Linguistic features :
    1. dictionaries
    2. Word-net
    3. Custom lists
    4. Gazetteer based recognition

Orthographic features :
    1. prefixes and suffixes of xi 
    2. conjunction of c and yi−1.
    3. all-digits
    4. alphanumeric
    5. Regular expressions
    
other features that can be used 

. Topic Feature 
. Word2Vec feature



If you can look up the whole document , then Context aggregation, Extended prediction history can be used
'''

def extract_features(sentence,index):
        
    #tokens in the window c = (xi−2, xi−1, xi , xi+1, xi+2) 
    token_in_window =[]
    token_in_window[0:5] = ['NA','NA','NA','NA','NA'] # 2 words preceeding current word and 2 words after current word
    pre_entity_1 = 'NA'
    pre_entity_2 = 'NA'
    
    if(index >= 11) :# this is a word which has two preceeding words to it in the sentence
        token_in_window[1] = sentence[index-7]
        token_in_window[0] = sentence[index-11]
        pre_entity_1 = sentence[index-4]
        pre_entity_2 = sentence[index-8]
    elif(index==3):# this is first word in the sentence
        pre_entity_1 = 'NA'
        pre_entity_2 = 'NA'
    elif(index==7):#this is second word in sentence
        token_in_window[1] = sentence[index-7]
        pre_entity_1 = sentence[index-4]
        pre_entity_2 = 'NA'

    token_in_window[2] = sentence[index-3] #current word
    
    sent_len = len(sentence) 
    
    if(index+3 >= sent_len) :# this is last word
        token_in_window[3] = 'NA'
    elif(index+7 >= sent_len):# this is second last word in the sentence
        token_in_window[3]= sentence[index+1]
    else:
        token_in_window[3]= sentence[index+1]
        token_in_window[4]= sentence[index+5]
    
    #get the POS and IOB tags for words in window
    
    pos_tags=[] 
    iob_tags=[]
    
    pos_tags[0:5] = ['NA','NA','NA','NA','NA']
    iob_tags[0:5] = ['NA','NA','NA','NA','NA']
    
    is_first_word_in_sentence = 0
    is_last_word_in_sentence = 0
    is_all_capitalized = 0 
    is_first_letter_capitalized = 0 

    if sentence[index-3].isupper():
        is_all_capitalized = 1
    
    is_first_letter_capitalized = is_first_letter_in_word_capitalized(sentence[index-3])
    
    if (index==3):#first word in sentence
        # POS for xi−2 , xi-1 will be NA
        pos_tags[2] = sentence[index-2]  #pos for xi
        iob_tags[2] =  sentence[index-1] #iob for xi
        
        pos_tags[3] = sentence[index+2] # POS for xi+1
        iob_tags[3] =  sentence[index+3] #iob for xi+1

        pos_tags[4] = sentence[index+6]# POS for xi+2
        iob_tags[4] =  sentence[index+7] #iob for xi+2
        
        is_first_word_in_sentence = 1

    elif ((index==sent_len-3) or (index==sent_len-2) ):    
        # POS for xi+1 , xi+2 will be NA
        pos_tags[0] = sentence[index-10]  #pos for xi-2
        iob_tags[0] =  sentence[index-9] #iob for xi-2
        
        pos_tags[1] = sentence[index-6] # POS for xi-1
        iob_tags[1] =  sentence[index-5] #iob for xi-1

        pos_tags[2] = sentence[index-2]# POS for xi
        iob_tags[2] =  sentence[index-1] #iob for xi

        is_last_word_in_sentence = 1
    
    elif(index+7 >= sent_len):# this is second last word in the sentence
        pos_tags[0] = sentence[index-10]  #pos for xi-2
        iob_tags[0] =  sentence[index-9] #iob for xi-2
        
        pos_tags[1] = sentence[index-6] # POS for xi-1
        iob_tags[1] =  sentence[index-5] #iob for xi-1

        pos_tags[2] = sentence[index-2]  #pos for xi
        iob_tags[2] =  sentence[index-1] #iob for xi
        
        pos_tags[3] = sentence[index+2] # POS for xi+1
        iob_tags[3] =  sentence[index+3] #iob for xi+1

    else:  
        pos_tags[0] = sentence[index-10]  #pos for xi-2
        iob_tags[0] =  sentence[index-9] #iob for xi-2
        
        pos_tags[1] = sentence[index-6] # POS for xi-1
        iob_tags[1] =  sentence[index-5] #iob for xi-1

        pos_tags[2] = sentence[index-2]  #pos for xi
        iob_tags[2] =  sentence[index-1] #iob for xi
        
        pos_tags[3] = sentence[index+2] # POS for xi+1
        iob_tags[3] =  sentence[index+3] #iob for xi+1

        pos_tags[4] = sentence[index+6]# POS for xi+2
        iob_tags[4] =  sentence[index+7] #iob for xi+2
          
    capitalization_pattern  = str(is_first_letter_in_word_capitalized (token_in_window[0]) )
    capitalization_pattern += str(is_first_letter_in_word_capitalized (token_in_window[1]) )
    capitalization_pattern += str(is_first_letter_in_word_capitalized (token_in_window[2]) )
    capitalization_pattern += str(is_first_letter_in_word_capitalized (token_in_window[3]) )
    capitalization_pattern += str(is_first_letter_in_word_capitalized (token_in_window[4]) )
    
    return {
        #'pre_word_1' : token_in_window[0],
        'pre_word_2' : token_in_window[1],
        'cur_word'   : token_in_window[2],
        'post_word_1': token_in_window[3],
        #'post_word_2': token_in_window[4],
        
        'pre_entity_1':pre_entity_1, # this is extended BIO tag
        'pre_entity_2':pre_entity_2,
        
        #'pos_tag_1': pos_tags[0],
        'pos_tag_2': pos_tags[1], 
        'pos_tag_3': pos_tags[2],
        'pos_tag_4': pos_tags[3], 
        #'pos_tag_5': pos_tags[4],
        
        #'bio_tag_1': iob_tags[0],
        'bio_tag_2': iob_tags[1],
        'bio_tag_3': iob_tags[2], 
        'bio_tag_4': iob_tags[3],
        #'bio_tag_5': iob_tags[4], 
        
        'fw': is_first_word_in_sentence,
        'lw': is_last_word_in_sentence,
        'acap': is_all_capitalized,
        'fcap': is_first_letter_capitalized,
        #'cap_ptrn':capitalization_pattern
       }       

In [480]:
'''
We need to take following out from the input sentence
    VB, VBD,VBG,VBZ, CC, CD, IN, NN, DT, NNS, PRP$,RB, JJ, RP, TO,PRP
even before supplying to classifier. 

This is needed becuase else we will end up in creating imbalanced data set with 100 time more entries for OTHER categories

Advance concept
===============
    It is mostly NNP for all entities but

    JJ and NNPS -  come in I-PER 
    JJ , 's , DT followed by JJ ( The Oval) - come in I-LOC 

    For I-ORG , Syntactic features seems important
        i.	Dependency path
        ii.	IOB chain
    Like The Test and County Criket Board, 
'''
def include_for_others(sentence,IOB_index):
    #took VBD I-VP O
    #Friday NNP I-NP O
    if sentence[IOB_index-2] == 'NNP' or sentence[IOB_index-2] == 'IN':
        return 1
    else:
        return 0

In [481]:
featuresets =[]
'''
The CONLL format is 
LEICESTERSHIRE NNP I-NP I-ORG
i.e 
<word> <POStag> <IOB tag> <IOB tag for Organizayion>

Hence i am parsing the sentence in similar way to extract features
'''
sentenceFeatures = []
tags = []
for sentence in sentences_with_words:
    featuresets = []
    sent_len = len(sentence) 
    num_word = (sent_len-1)/4

    if(num_word < 3): #in some case the sentence is of just 1 or 2 words. I am avoiding those sentences for now
        continue
    count = 0
    while (count < num_word):
        IOB_index = (count * 4 )+ 3
        if ( sentence[IOB_index] == 'I-ORG'):
            featuresets.append( (extract_features(sentence,IOB_index) , 'ORG') )
        elif ( sentence[IOB_index] == 'I-LOC'):
            featuresets.append( (extract_features(sentence,IOB_index) , 'LOC') )
        elif ( sentence[IOB_index] == 'I-PER'):
            featuresets.append( (extract_features(sentence,IOB_index) , 'PER') )
        elif ( sentence[IOB_index] == 'I-MISC'):
            featuresets.append( (extract_features(sentence,IOB_index) , 'MISC') )
        elif include_for_others(sentence,IOB_index):
            featuresets.append( (extract_features(sentence,IOB_index) , 'OTHER') )
 
        count = count + 1
    sentenceFeatures.append(featuresets)
    
print len(sentenceFeatures)
#print featuresets
#Sample feature set should be list of tuples which should look like 
#[({'ed': 4, 'rel_type': 'PL', 'ps2': 0, 'ps1': 0, 'e1': 'Person', 'e2': 'Location'}, 'positive'), 
#({'ed': 1, 'rel_type': 'CL', 'ps2': 0, 'ps1': 0, 'e1': 'Company', 'e2': 'Location'}, 'positive'), 
#({'ed': 2, 'rel_type': 'PL', 'ps2': 0, 'ps1': 1, 'e1': 'Person', 'e2': 'Location'}, 'negative')]

2091


In [482]:
%%time
number_of_sample_to_take_crf = len(sentenceFeatures)
partition = 2

print len(sentenceFeatures[0])
crftrain_set, crftest_set = sentenceFeatures[0:number_of_sample_to_take_crf/partition], sentenceFeatures[number_of_sample_to_take_crf/partition + 1:]
print len(sentenceFeatures)
print len(crftrain_set)


X_train=[[]]
Y_train = [[]]

length = len(crftrain_set)
for sent in crftrain_set[0:length]:
    #print sent
    temp1 = []
    temp2 = []
    for word in sent:
        temp1.append(word[0])
        temp2.append(word[1])
    X_train.append(temp1)
    Y_train.append(temp2)
    
    
    temp1 = []
    temp2 = []



8
2091
1045
Wall time: 29 ms


In [483]:
X_test=[[]]
Y_test = [[]]

lengthtest = len(crftest_set)
print lengthtest

for sent in crftest_set[0:lengthtest]:
    #print sent
    temp1 = []
    temp2 = []
    for word in sent:
        temp1.append(word[0])
        temp2.append(word[1])
    X_test.append(temp1)
    Y_test.append(temp2)
    
    temp1 = []
    temp2 = []

1045


In [484]:
print Y_test[6]

['PER', 'PER']


In [485]:
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(X_train, Y_train):
    trainer.append(xseq, yseq)
    

In [486]:
%%time
trainer.train('conll2002-esp.crfsuite')

Wall time: 585 ms


In [487]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.logparser.last_iteration
print len(trainer.logparser.iterations), trainer.logparser.iterations[-1]

117 {'loss': 1809.927857, 'error_norm': 0.833546, 'linesearch_trials': 1, 'active_features': 7361, 'num': 117, 'time': 0.004, 'scores': {}, 'linesearch_step': 1.0, 'feature_norm': 23.80522}


In [488]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')


In [489]:
print tagger.tag(X_test[1])
print Y_test[1]

['LOC', 'MISC', 'OTHER', 'OTHER']
['LOC', 'MISC', 'OTHER', 'OTHER']


In [490]:
from nltk.metrics import ConfusionMatrix
Y_pred=[]
for row in X_test:
    Y_pred.append(tagger.tag(row))
print len(Y_pred)
new_y_Pred=[]
new_y_Test=[]
for row in Y_test:
    new_y_Test=new_y_Test+row

for row in Y_pred:
    new_y_Pred=new_y_Pred+row
print len(new_y_Test)
#Y_pred = tagger.tag(X_test[1])
cm = ConfusionMatrix(new_y_Test, new_y_Pred)
print cm


1046
6847
      |                   O      |
      |         M         T      |
      |    L    I    O    H    P |
      |    O    S    R    E    E |
      |    C    C    G    R    R |
------+--------------------------+
  LOC | <742>  32   68   58   87 |
 MISC |   39 <496>  59   32   30 |
  ORG |   82   23 <890>  29   50 |
OTHER |  121   23   46<2429>  52 |
  PER |   48   10   18   17<1366>|
------+--------------------------+
(row = reference; col = test)



In [491]:
from collections import Counter
true_positives = Counter()
false_negatives = Counter()
false_positives = Counter()


In [492]:

labels = set('LOC MISC ORG OTHER PER'.split())
for i in labels:
    for j in labels:
        if i == j:
            true_positives[i] += cm[i,j]
        else:
            false_negatives[i] += cm[i,j]
            false_positives[j] += cm[i,j]

print "TP:", sum(true_positives.values()), true_positives
print "FN:", sum(false_negatives.values()), false_negatives
print "FP:", sum(false_positives.values()), false_positives

print "Accuracy", float(sum(true_positives.values()))*100/(sum(true_positives.values()) + sum(false_negatives.values()) + sum(false_positives.values()))
print 

print "Fscore for each label"
for i in sorted(labels):
    if true_positives[i] == 0:
        fscore = 0
    else:
        precision = true_positives[i] / float(true_positives[i]+false_positives[i])
        recall = true_positives[i] / float(true_positives[i]+false_negatives[i])
        fscore = 2 * (precision * recall) / float(precision + recall)
    print i, fscore


TP: 5923 Counter({'OTHER': 2429, 'PER': 1366, 'ORG': 890, 'LOC': 742, 'MISC': 496})
FN: 924 Counter({'LOC': 245, 'OTHER': 242, 'ORG': 184, 'MISC': 160, 'PER': 93})
FP: 924 Counter({'LOC': 290, 'PER': 219, 'ORG': 191, 'OTHER': 136, 'MISC': 88})
Accuracy 76.2192767984

Fscore for each label
LOC 0.735017335315
MISC 0.8
ORG 0.825986078886
OTHER 0.927807486631
PER 0.897503285151
